<a href="https://colab.research.google.com/github/leeds1219/MHR-TMS-SD-Agent-Prototype/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LG Pilot Demo

In [2]:
pip install transformers torch PyMuPDF python-docx python-pptx konlpy rank-bm25

   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 24.1/24.1 MB 105.4 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 253.0/253.0 kB 25.1 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 472.8/472.8 kB 45.4 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 19.4/19.4 MB 124.5 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 495.9/495.9 kB 42.1 MB/s eta 0:00:00
   ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 175.3/175.3 kB 15.7 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import os
from typing import Dict, List, Any

# PDF Ï≤òÎ¶¨
import fitz
# DOCX Ï≤òÎ¶¨
from docx import Document
# PPTX Ï≤òÎ¶¨
from pptx import Presentation
from konlpy.tag import Okt
from rank_bm25 import BM25Okapi

# ==============================================================================
# 1. Î™®Îç∏/ÌÜ†ÌÅ¨ÎÇòÏù¥Ï†Ä Ï¥àÍ∏∞Ìôî Î∞è ÌôòÍ≤Ω ÏÑ§Ï†ï
# ==============================================================================

model_name = "LGAI-EXAONE/EXAONE-4.0-1.2B"
model = None
tokenizer = None
okt = Okt() # KonLPy Okt Ï¥àÍ∏∞Ìôî

def initialize_llm(model_name: str):
    """LLM Î™®Îç∏Í≥º ÌÜ†ÌÅ¨ÎÇòÏù¥Ï†ÄÎ•º Î°úÎìúÌïòÍ≥† Ï†ÑÏó≠ Î≥ÄÏàòÏóê Ìï†ÎãπÌï©ÎãàÎã§."""
    global model, tokenizer
    try:
        print(f"Loading model: {model_name}...")
        # ‚≠ê GPU ÌôòÍ≤ΩÏóêÏÑú Ïã§ÌñâÏùÑ ÏúÑÌï¥ device_map="auto"ÏôÄ dtype="bfloat16" ÏÇ¨Ïö©
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            dtype=torch.bfloat16,
            device_map="auto"
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        print("Model and Tokenizer loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")
        model, tokenizer = None, None
        print("FATAL: Model loading failed. LLM generation will be impossible.")

def tokenize_ko_en(text: str) -> List[str]:
    """ÌïúÍ∏ÄÏùÄ ÌòïÌÉúÏÜå Î∂ÑÏÑù(Ïä§ÌÖåÎ∞ç Ìè¨Ìï®), ÏòÅÏñ¥Îäî ÏÜåÎ¨∏ÏûêÌôî Î∞è Í≥µÎ∞± Î∂ÑÎ¶¨ÌïòÏó¨ ÌÜ†ÌÅ∞ Î∞òÌôò"""
    # ÌïúÍ∏Ä ÌòïÌÉúÏÜå Î∂ÑÏÑù
    ko_tokens = okt.morphs(text, stem=True)

    # ÏòÅÏñ¥ Î∞è Í∏∞ÌÉÄ ÌÖçÏä§Ìä∏ Ï≤òÎ¶¨ (ÌòïÌÉúÏÜå Î∂ÑÏÑù Í≤∞Í≥ºÏóê Ìè¨Ìï®ÎêòÏßÄ ÏïäÏùÄ ÌÜ†ÌÅ∞)
    text_words = set(text.split())

    en_tokens = [
        token.lower()
        for token in text_words
        if token.lower() not in ko_tokens and token.isalnum() # ÏàúÏàò ÏòÅÏñ¥ Îã®Ïñ¥ Ï∂îÏ∂ú Ïãú Ïú†Ìö®ÏÑ± Í≤ÄÏÇ¨ Ï∂îÍ∞Ä
    ]
    return ko_tokens + en_tokens

In [4]:
# ==============================================================================
# 2. Î¨∏ÏÑú Ï≤òÎ¶¨ Ìï®Ïàò
# ==============================================================================

def parse_pdf(file_path: str) -> List[Dict]:
    """PDF ÌååÏùºÏùÑ ÌååÏã±ÌïòÏó¨ ÌéòÏù¥ÏßÄÎ≥Ñ ÌÖçÏä§Ìä∏Î•º Î¶¨Ïä§Ìä∏Î°ú Î∞òÌôòÌï©ÎãàÎã§."""
    pages_data = []
    try:
        doc = fitz.open(file_path)
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text = page.get_text("text")
            if text.strip():
                pages_data.append({'type': 'page', 'index': page_num + 1, 'content': text.strip()})
        doc.close()
    except Exception as e:
        print(f"PDF Parsing Error in {file_path}: {e}")
    return pages_data

def parse_pptx(file_path: str) -> List[Dict]:
    """PPTX ÌååÏùºÏùÑ ÌååÏã±ÌïòÏó¨ Ïä¨ÎùºÏù¥ÎìúÎ≥Ñ ÌÖçÏä§Ìä∏Î•º Î¶¨Ïä§Ìä∏Î°ú Î∞òÌôòÌï©ÎãàÎã§."""
    slides_data = []
    try:
        prs = Presentation(file_path)
        for slide_num, slide in enumerate(prs.slides):
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text") and shape.has_text_frame:
                    slide_text.append(shape.text)

            content = "\n".join(t.strip() for t in slide_text if t.strip())
            if content:
                 slides_data.append({'type': 'slide', 'index': slide_num + 1, 'content': content})
    except Exception as e:
        print(f"PPTX Parsing Error in {file_path}: {e}")
    return slides_data

def parse_docx(file_path: str) -> List[Dict]:
    """DOCX ÌååÏùºÏùÑ ÌååÏã±ÌïòÏó¨ Îã®ÎùΩÎ≥Ñ ÌÖçÏä§Ìä∏Î•º Î¶¨Ïä§Ìä∏Î°ú Î∞òÌôòÌï©ÎãàÎã§."""
    paragraphs_data = []
    try:
        doc = Document(file_path)
        for para_num, paragraph in enumerate(doc.paragraphs):
            text = paragraph.text.strip()
            if text:
                paragraphs_data.append({'type': 'paragraph', 'index': para_num + 1, 'content': text})
    except Exception as e:
        print(f"DOCX Parsing Error in {file_path}: {e}")
    return paragraphs_data

def parse_folder_documents(folder_path: str) -> List[Dict[str, Any]]:
    """Ìè¥Îçî ÎÇ¥Ïùò ÏßÄÏõêÎêòÎäî Î¨∏ÏÑú ÌååÏùºÎì§ÏùÑ ÌååÏã±ÌïòÏó¨ 'Î¨∏ÏÑú Í∞ùÏ≤¥ Î¶¨Ïä§Ìä∏' ÌòïÌÉúÎ°ú Î∞òÌôòÌï©ÎãàÎã§."""
    all_documents_list: List[Dict[str, Any]] = []
    if not os.path.exists(folder_path):
        print(f"Error: Folder path does not exist: {folder_path}")
        return []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            extension = filename.split('.')[-1].lower()
            parsed_data: List[Dict] = []

            try:
                if extension == 'pdf':
                    parsed_data = parse_pdf(file_path)
                elif extension == 'pptx':
                    parsed_data = parse_pptx(file_path)
                elif extension == 'docx':
                    parsed_data = parse_docx(file_path)

                if parsed_data:
                    # Î¨∏ÏÑú Îã®ÏúÑ Í∞ùÏ≤¥Î°ú Íµ¨ÏÑ±
                    all_documents_list.append({
                        'filename': filename,
                        'passages': parsed_data
                    })
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    return all_documents_list

In [5]:
# ==============================================================================
# 3. RAG/BM25 Î∞è LLM Í¥ÄÎ†® Ìï®Ïàò
# ==============================================================================

def call_llm_generate(prompt: str, max_new_tokens: int) -> str:
    """LLMÏùÑ Ìò∏Ï∂úÌïòÍ≥† ÏùëÎãµ ÌÖçÏä§Ìä∏Îßå Î∞òÌôòÌï©ÎãàÎã§. Î™®Îç∏ Î°úÎìú Ïã§Ìå® Ïãú ÏòàÏô∏Î•º Î∞úÏÉùÏãúÌÇµÎãàÎã§."""
    global model, tokenizer
    if model is None or tokenizer is None:
        raise RuntimeError("FATAL ERROR: LLM Model or Tokenizer failed to load. Cannot perform generation.")

    messages = [{"role": "user", "content": prompt}]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device) # ÏûÖÎ†• ÌÖêÏÑúÎ•º Î™®Îç∏ ÎîîÎ∞îÏù¥Ïä§Î°ú Ïù¥Îèô

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    response_raw = tokenizer.decode(output[0], skip_special_tokens=True)

    # [|assistant|] Îí§Ïùò ÏÉùÏÑ±Îêú ÌÖçÏä§Ìä∏Îßå Î∞òÌôò
    assistant_tag = "[|assistant|]"
    if assistant_tag in response_raw:
        return response_raw
    else:
        # Î™®Îç∏Ïù¥ ÌîÑÎ°¨ÌîÑÌä∏Îßå Î∞òÌôòÌïòÎäî Í≤ΩÏö∞, Ï†ÑÏ≤¥Î•º Î∞òÌôò
        return response_raw

def retrieve_and_augment_by_file(query: str, file_passages: List[Dict], file_name: str, N: int = 5):
    """
    ÌäπÏ†ï ÌååÏùº ÎÇ¥Ïùò Ï°∞Í∞Å(Passages)Ïóê ÎåÄÌï¥ÏÑúÎßå BM25 Í≤ÄÏÉâÏùÑ ÏàòÌñâÌïòÍ≥† 1Ï∞® ÌîÑÎ°¨ÌîÑÌä∏Î•º Íµ¨ÏÑ±Ìï©ÎãàÎã§.
    """
    if not file_passages:
        return f"Î¨∏ÏÑú '{file_name}'ÏóêÎäî ÎÇ¥Ïö©Ïù¥ ÏóÜÏäµÎãàÎã§.", None, None

    # ÏΩîÌçºÏä§ ÌÜ†ÌÅ∞Ìôî Î∞è BM25 Ïù∏Îç±Ïä§ ÏÉùÏÑ±
    tokenized_corpus = [tokenize_ko_en(p['content']) for p in file_passages]
    bm25 = BM25Okapi(tokenized_corpus)

    # ÏßàÎ¨∏ ÌÜ†ÌÅ∞Ìôî Î∞è Ï†êÏàò Í≥ÑÏÇ∞
    tokenized_query = tokenize_ko_en(query)
    doc_scores = bm25.get_scores(tokenized_query)

    # Í¥ÄÎ†®ÏÑ± ÎÜíÏùÄ ÏÉÅÏúÑ NÍ∞ú Ïù∏Îç±Ïä§ Ï∂îÏ∂ú
    top_n_indices = doc_scores.argsort()[::-1][:N]

    print("-" * 50)
    print(f"üîé BM25 Í≤ÄÏÉâ ÏãúÏûë (Î¨∏ÏÑú: {file_name})")
    print("-" * 50)

    # Î¨∏ÏÑú ÎÇ¥Ïö© Ìè¨Îß∑ÌåÖ
    document_parts = []

    for rank_idx, doc_index in enumerate(top_n_indices):
        passage_info = file_passages[doc_index]
        doc_content = passage_info['content'].replace('\n', ' ')
        document_parts.append(f"Î≤àÌò∏ {rank_idx + 1}. : {doc_content}")
        # ÎîîÎ≤ÑÍπÖÏö© Ï∂úÎ†•ÏùÄ ÏÉùÎûµÌïòÍ≥† ÌîÑÎ°¨ÌîÑÌä∏Ïóê ÏßëÏ§ë

    document_formatted = "\n".join(document_parts)

    # 1Ï∞® ÌîÑÎ°¨ÌîÑÌä∏ ÌÖúÌîåÎ¶ø: Í∞ÄÏû• Í¥ÄÎ†® ÍπäÏùÄ Î¨∏ÏÑú Î≤àÌò∏ ÏÑ†ÌÉù Ïú†ÎèÑ
    prompt_first_turn = f"""ÏÉÅÌô©: {query}
{document_formatted}

Ï†Ä Î¨∏ÏÑúÎì§(Î≤àÌò∏ 1~{len(top_n_indices)}) Ï§ë ÌòÑ ÏÉÅÌô©Í≥º Í∞ÄÏû• Í¥ÄÎ†®Ïù¥ ÍπäÏùÄ Î¨∏ÏÑúÏùò **Î≤àÌò∏ Î≤àÌò∏(Ïà´Ïûê)**Îßå ÎãµÎ≥ÄÌï¥ Ï§ò."""

    return prompt_first_turn, top_n_indices, file_name

def select_document_and_ask_sufficient_action(query: str, model_response_rank_raw: str, file_passages: List[Dict], top_n_indices: List[int], selected_doc_title: str) -> str:
    """
    Î™®Îç∏ ÏùëÎãµÏóêÏÑú Î≤àÌò∏Î•º Ï∂îÏ∂úÌïòÏó¨ Î¨∏ÏÑúÎ•º ÏÑ†ÌÉùÌïòÍ≥†, Ï°∞ÏπòÏùò Ï∂©Î∂ÑÏÑ±ÏùÑ Î¨ªÎäî 2Ï∞® ÌîÑÎ°¨ÌîÑÌä∏Î•º ÏÉùÏÑ±Ìï©ÎãàÎã§.
    """
    # 1. Î™®Îç∏ ÏùëÎãµÏóêÏÑú Î≤àÌò∏ Ï∂îÏ∂ú
    response_content = model_response_rank_raw.split('[|assistant|]')[-1]
    rank_match = re.search(r'\d+', response_content)
    selected_rank = int(rank_match.group(0)) if rank_match else 1

    # 2. ÏÑ†ÌÉùÎêú Î¨∏ÏÑú Ï†ïÎ≥¥ Í∞ÄÏ†∏Ïò§Í∏∞ (Ïú†Ìö®ÏÑ± Í≤ÄÏÇ¨ Î∞è Ïù∏Îç±Ïã±)
    if selected_rank > len(top_n_indices) or selected_rank <= 0:
        print(f"Í≤ΩÍ≥†: ÏÑ†ÌÉùÎêú Î≤àÌò∏ Î≤àÌò∏({selected_rank})Í∞Ä Ïú†Ìö®ÌïòÏßÄ ÏïäÏïÑ 1ÏúÑ Î¨∏ÏÑúÎ°ú ÎåÄÏ≤¥Ìï©ÎãàÎã§.")
        selected_rank = 1

    try:
        selected_doc_index = top_n_indices[selected_rank - 1]
        selected_passage = file_passages[selected_doc_index]
        selected_content_type = selected_passage['type']
        selected_index = selected_passage['index']
        selected_doc_content = selected_passage['content'].replace('\n', ' ')
    except IndexError:
        # Îß§Ïö∞ ÎìúÎ¨∏ ÏºÄÏù¥Ïä§, Ïù¥ÎØ∏ ÏúÑÏóêÏÑú Ï≤òÎ¶¨ÌñàÏúºÎÇò ÏïàÏ†ÑÏùÑ ÏúÑÌï¥ Ï∂îÍ∞Ä
        print("ÏπòÎ™ÖÏ†Å Ïò§Î•ò: top_n_indices Ïù∏Îç±Ïã± Ïã§Ìå®. Ï≤´ Î≤àÏß∏ Î¨∏ÏÑúÎ°ú ÎåÄÏ≤¥.")
        selected_passage = file_passages[top_n_indices[0]]
        selected_content_type = selected_passage['type']
        selected_index = selected_passage['index']
        selected_doc_content = selected_passage['content'].replace('\n', ' ')

    # 3. 2Ï∞® ÌîÑÎ°¨ÌîÑÌä∏ ÌÖúÌîåÎ¶ø Ï†ïÏùò Î∞è Ìè¨Îß∑: Ï°∞Ïπò ÎãµÎ≥Ä ÏÉùÏÑ±
    doc_identifier = f"{selected_content_type} {selected_index}"

    prompt_second_turn = f"""
ÏÉÅÌô©: {query}

--- ÏÑ†ÌÉùÎêú Î¨∏ÏÑú ({selected_doc_title}, {doc_identifier}) ---
{selected_doc_content}
--------------------------------

ÏúÑ ÏÑ†ÌÉùÎêú Î¨∏ÏÑúÎ•º Ï∞∏Í≥†ÌïòÏó¨, ÌòÑ ÏÉÅÌô©Ïóê ÎåÄÌï¥ **Í∞ÄÏû• Ï†ÅÏ†àÌïú Ï°∞Ïπò**Î•º ÏßßÍ≤å ÎãµÎ≥ÄÌï¥ Ï§ò."""

    return prompt_second_turn

def create_rewriting_prompt(full_response: str) -> str:
    """Îã§Ïùå Í≤ÄÏÉâÏóê ÏµúÏ†ÅÌôîÎêú ÏøºÎ¶¨Î•º ÏÉùÏÑ±ÌïòÎäî ÌîÑÎ°¨ÌîÑÌä∏Î•º ÎßåÎì≠ÎãàÎã§."""
    rewriting_prompt = f"""
Îã§ÏùåÏùÄ Ïù¥Ï†Ñ Îã®Í≥ÑÏóêÏÑú 'Î∂àÏ∂©Î∂Ñ'ÌïòÎã§Í≥† ÌåêÎã®Îêú ÏµúÏ¢Ö Ï°∞Ïπò Í≤∞Í≥ºÏûÖÎãàÎã§.

--- Ïù¥Ï†Ñ Ï°∞Ïπò ÎãµÎ≥Ä ---
{full_response}
--------------------

Ïù¥ ÎãµÎ≥ÄÏùò ÎÇ¥Ïö©ÏùÑ Í∏∞Î∞òÏúºÎ°ú, **ÌòÑ ÏÉÅÌô©ÏùÑ ÏôÑÏ†ÑÌûà Ìï¥Í≤∞ÌïòÍ∏∞ ÏúÑÌï¥ Îã§Ïùå ÌÑ¥Ïóê BM25 Í≤ÄÏÉâÏùÑ ÏàòÌñâÌï† Ïö©ÎèÑÏùò Í∞ÄÏû• ÌïµÏã¨Ï†ÅÏù¥Í≥† Íµ¨Ï≤¥Ï†ÅÏù∏ ÏßàÎ¨∏ 1Í∞ú**Îßå ÏÉùÏÑ±Ìï¥ Ï£ºÏÑ∏Ïöî. ÏßàÎ¨∏ÏùÄ 10Îã®Ïñ¥ Ïù¥ÎÇ¥Î°ú Í∞ÑÍ≤∞Ìï¥Ïïº Ìï©ÎãàÎã§. (Ïòà: 'ÏïïÏ∂ïÍ∏∞ ÍµêÏ≤¥ ÌõÑ Ï∂îÍ∞Ä Ï†êÍ≤Ä ÏÇ¨Ìï≠')
"""
    return rewriting_prompt

In [ ]:
# ==============================================================================
# 4. Ï¥àÍ∏∞ ÏÑ§Ï†ï Î∞è ÏàúÏ∞®Ï†Å RAG Î£®ÌîÑ Ïã§Ìñâ
# ==============================================================================

if __name__ == "__main__":
    # 1. LLM Ï¥àÍ∏∞Ìôî
    initialize_llm(model_name)

    if model is None:
        exit() # Î™®Îç∏ Î°úÎìú Ïã§Ìå® Ïãú Ï¢ÖÎ£å

    # 2. Î¨∏ÏÑú ÌååÏã± ÏÑ§Ï†ï
    documents_folder_path = "/content"
    documents_list = parse_folder_documents(documents_folder_path)

    if not documents_list:
        print("FATAL ERROR: No documents loaded. Please ensure files exist at the path.")
        exit()

    # Î¨∏ÏÑú ÏàúÏÑú Ï†ïÏùò
    DOCUMENT_ORDER = [
        'Î¨∏ÏÑú_ÏÉòÌîå_1.pdf',
        'Î¨∏ÏÑú_ÏÉòÌîå_2.pdf',
        'Î¨∏ÏÑú_ÏÉòÌîå_3.pdf',
    ]
    MAX_ITERATIONS = len(DOCUMENT_ORDER)

    # Ï¥àÍ∏∞ ÏøºÎ¶¨ ÏÑ§Ï†ï
    initial_query = """CH21 ÏóêÎü¨ Î∞úÏÉùÏúºÎ°ú Ïã§ÎÇ¥Í∏∞ Ïö¥Ï†Ñ Î∂àÍ∞Ä ÏóêÎü¨ ÏµúÏ¥àÎ∞úÏÉùÏãúÏ†ê : 2025.06.03, ÏóêÎü¨ ÏµúÏã†Î∞úÏÉùÏãúÏ†ê : 2025.07.08\nÎàÑÏ†ÅÎ∞úÏÉùÏùºÏàò : 33, ch21 : 106, ch26: 0, ch29 :0, Î∞±ÏóÖinv1 : Ï†ïÏÉÅ, Î∞±ÏóÖinv2 : Ï†ïÏÉÅ"""
    initial_query = input()
    current_query = initial_query
    iteration_history = []

    print(f"\nüöÄ Initial Query: {initial_query}")

    # 3. ÏàúÏ∞®Ï†Å RAG Î£®ÌîÑ Ïã§Ìñâ
    for i, doc_name_to_search in enumerate(DOCUMENT_ORDER):
        print("\n" + "#"*80)
        print(f"üîÑ ITERATION {i+1}/{MAX_ITERATIONS}: Î¨∏ÏÑú '{doc_name_to_search}' Í≤ÄÏÉâ ÏãúÏûë")
        print(f" ¬† ÌòÑÏû¨ Í≤ÄÏÉâ ÏøºÎ¶¨: {current_query}")
        print("#"*80)

        # [Î¨∏ÏÑú Îã®ÏúÑ Ï≤òÎ¶¨] Î¨∏ÏÑú Í∞ùÏ≤¥ Ï∞æÍ∏∞
        document_unit: Dict[str, Any] = next((doc for doc in documents_list if doc['filename'] == doc_name_to_search), {})

        if not document_unit or not document_unit.get('passages'):
            print(f"Warning: Document '{doc_name_to_search}' not found or has no content. Skipping.")
            continue

        file_passages: List[Dict] = document_unit['passages']

        # A. 1Ï∞® Îã®Í≥Ñ: BM25 Í≤ÄÏÉâ Î∞è Î¨∏ÏÑú Î≤àÌò∏ ÏÑ†ÌÉù ÌîÑÎ°¨ÌîÑÌä∏ ÏÉùÏÑ±
        prompt_to_get_rank, top_n_indices, selected_doc_title = retrieve_and_augment_by_file(
            query=current_query,
            file_passages=file_passages,
            file_name=doc_name_to_search,
            N=5
        )

        # B. 1Ï∞® Î™®Îç∏ Ìò∏Ï∂ú: Í¥ÄÎ†® Î¨∏ÏÑú Î≤àÌò∏(Ïà´Ïûê) ÎãµÎ≥Ä Î∞õÍ∏∞
        try:
            rank_response_raw = call_llm_generate(prompt_to_get_rank, max_new_tokens=1)
            rank_response = rank_response_raw.split('[|assistant|]')[-1].strip()
        except RuntimeError as e:
            print(f"LLM Call Error: {e}")
            break

        print(f"\nüí° LLM ÏÑ†ÌÉù Î≤àÌò∏: {rank_response}")

        # C. 2Ï∞® Îã®Í≥Ñ: ÏÑ†ÌÉùÎêú Î¨∏ÏÑúÎ°ú ÏµúÏ¢Ö Ï°∞Ïπò ÌîÑÎ°¨ÌîÑÌä∏ ÏÉùÏÑ±
        prompt_to_get_action = select_document_and_ask_sufficient_action(
            query=current_query,
            model_response_rank_raw=rank_response_raw,
            file_passages=file_passages,
            top_n_indices=top_n_indices,
            selected_doc_title=selected_doc_title
        )

        # D. 2Ï∞® Î™®Îç∏ Ìò∏Ï∂ú: ÏµúÏ¢Ö Ï°∞Ïπò ÎãµÎ≥Ä Î∞õÍ∏∞
        try:
            action_response_raw = call_llm_generate(prompt_to_get_action, max_new_tokens=128)
            action_response = action_response_raw.split('[|assistant|]')[-1].strip()
        except RuntimeError as e:
            print(f"LLM Call Error: {e}")
            break

        print("\n" + "="*80)
        print(f"‚û°Ô∏è ITERATION {i+1} ÎãµÎ≥Ä: {action_response}")
        print("="*80)

        # E. ÌûàÏä§ÌÜ†Î¶¨ Ï†ÄÏû• Î∞è Îã§Ïùå ÌÑ¥ Ï§ÄÎπÑ
        iteration_history.append({
            'turn': i + 1,
            'doc_name': doc_name_to_search,
            'query_for_search': current_query,
            'action_response': action_response
        })

        is_last_iteration = (i + 1) == MAX_ITERATIONS

        if is_last_iteration:
            print("\n‚úÖ Î™®Îì† Î¨∏ÏÑúÎ•º Í≤ÄÏÉâ ÏôÑÎ£åÌñàÏäµÎãàÎã§. Î∞òÎ≥µÏùÑ Ï¢ÖÎ£åÌï©ÎãàÎã§.")
            break

        # Îã§Ïùå ÌÑ¥ÏùÑ ÏúÑÌïú ÏÉàÎ°úÏö¥ Í≤ÄÏÉâ ÏøºÎ¶¨ ÏÉùÏÑ±
        rewriting_prompt = create_rewriting_prompt(action_response)
        print("\nüîç ÏÉàÎ°úÏö¥ Í≤ÄÏÉâ ÏøºÎ¶¨ ÏÉùÏÑ±ÏùÑ ÏúÑÌï¥ LLM Ìò∏Ï∂ú...")
        try:
            # ÏÉàÎ°úÏö¥ ÏøºÎ¶¨ ÏÉùÏÑ± (max_new_tokens=20)
            new_query_response_raw = call_llm_generate(rewriting_prompt, max_new_tokens=20)
            # Î™®Îç∏ ÏùëÎãµÏóêÏÑú ÏßàÎ¨∏ Î∂ÄÎ∂ÑÎßå ÍπîÎÅîÌïòÍ≤å Ï∂îÏ∂ú
            current_query = new_query_response_raw.split("</think>")[-1].split('[|assistant|]')[-1].strip()
        except RuntimeError as e:
            print(f"LLM Call Error during query rewriting: {e}")
            break

        print(f"ÏÉàÎ°úÏö¥ Í≤ÄÏÉâ ÏøºÎ¶¨: {current_query}")

Loading model: LGAI-EXAONE/EXAONE-4.0-1.2B...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Model and Tokenizer loaded successfully.
FATAL ERROR: No documents loaded. Please ensure files exist at the path.
